# QAOA Portfolio Optimization with Parallel Multi-Start Optimization and DWE

This notebook implements a **Parallel Multi-Start Optimization** strategy for QAOA parameters in portfolio optimization using Domain Wall Encoding (DWE).

Instead of a Genetic Algorithm, this approach runs multiple independent local optimizations in parallel, each starting from a different random initial point. This is often more efficient for finding good solutions to non-convex problems like QAOA on local machines.

The DWE technique is used to handle the Hamming-weight constraint as a penalty within the objective function.

**Key components:**
- **`qaoa_parallel_optimizer_worker.py`**: A separate Python file containing the `run_single_optimization` function, which performs one local QAOA optimization run using a SciPy optimizer (e.g., COBYLA) and Qiskit's `Sampler` for shot-based simulation. This separation is crucial for robust multiprocessing.
- **`concurrent.futures.ProcessPoolExecutor`**: Used to manage and execute the `run_single_optimization` calls in parallel across available CPU cores.
-

## 1. Setup and Imports

We'll import necessary libraries, including `concurrent.futures` for parallel processing and our `run_single_optimization` function.

In [1]:
import numpy as np
import os
import time
import concurrent.futures # For parallel processing

# Import the single optimization worker function
from qaoa_parallel_optimizer_worker import run_single_optimization

# For analysis of optimal bitstring
from qiskit.circuit import ParameterVector
from qiskit.primitives import Sampler
from qokit.qaoa_circuit_portfolio import get_parameterized_qaoa_circuit

## 2. Define Problem Parameters (with DWE)

This section defines your portfolio optimization problem with DWE, creating the `po_problem` dictionary that will be passed to each parallel optimization run.

In [2]:
import numpy as np
from multiprocessing import Pool
import os
import time
from qokit.qaoa_circuit_portfolio import get_parameterized_qaoa_circuit
import json
from qiskit.circuit import QuantumCircuit
from qiskit import transpile
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt


# --- Define Parallelization and QAOA Parameters ---
num_parallel_runs = 10 # Number of parallel multi-start runs
p = 100 # Number of QAOA layers (depth)
max_iterations_optimizer = 2200 # Max iterations for the classical optimizer

num_shots_simulator = 256 # Number of shots for quantum circuit simulation

# --- Define Problem Parameters (aligned with your DWE notebook) ---
N = 5
K = int(N * 0.4)

# --- Generate random mu and Sigma for larger N ---
np.random.seed(42) # for reproducibility of the problem definition
mu = np.random.uniform(0.05, 0.20, N)
Sigma = np.random.uniform(0.001, 0.015, (N, N))
Sigma = (Sigma + Sigma.T) / 2 # Make it symmetric
Sigma = Sigma + np.eye(N) * 0.005 # Add a small diagonal component for stability

q = 0.5 # Risk aversion parameter
lambda_sum = 0 # DWE-inspired penalty coefficient for sum constraint

# --- Calculate Coefficients for Total Cost Hamiltonian (H_C = H_C^objective + H_C^DWE-penalty) ---
factor_J_obj = (2 * q) / (K**2)
factor_h_linear_obj = -1 / K
factor_h_diagonal_obj = q / (K**2)

J_coeffs_objective = {}
h_coeffs_objective = {}

for i in range(N):
    for j in range(i + 1, N):
        J_coeffs_objective[(i, j)] = factor_J_obj * Sigma[i, j]

for i in range(N):
    h_coeffs_objective[i] = factor_h_linear_obj * mu[i] + factor_h_diagonal_obj * Sigma[i, i]

J_coeffs_total = {}
h_coeffs_total = {}

for (i, j), val in J_coeffs_objective.items():
    J_coeffs_total[(i, j)] = val + 2 * lambda_sum

for i, val in h_coeffs_objective.items():
    h_coeffs_total[i] = val - 5 * lambda_sum

# Construct the custom po_problem dictionary with DWE coefficients
po_problem = {
    "N": N,
    "K": K,
    "q": q,
    "J": J_coeffs_total,
    "h": h_coeffs_total,
    "means": mu,
    "cov": Sigma,
    "q_orig": q,
    "scale": 1.0
}

print(f"--- Problem Parameters & Custom QUBO Defined for N={N} ---")
print(f"Number of assets (N): {N}")
print(f"Assets to select (K): {K}")
print(f"QAOA Layers (p): {p}")
print(f"DWE Penalty Lambda_sum: {lambda_sum}")
print(f"Optimizer Max Iterations: {max_iterations_optimizer}")
print(f"Simulator Shots: {num_shots_simulator}")

# --- Display Calculated QUBO Coefficients ---
print("\n--- Calculated QUBO Coefficients ---")
print("\nObjective Hamiltonian (H_C^objective) Coefficients:")
print("  J_obj (Quadratic Terms):")
print(json.dumps({str(k): f"{v:.6f}" for k, v in J_coeffs_objective.items()}, indent=2)[:500] + "...") # Limit output size
print("  h_obj (Linear Terms):")
print(json.dumps({str(k): f"{v:.6f}" for k, v in h_coeffs_objective.items()}, indent=2))

print("\nFull Cost Hamiltonian (H_C = H_C^objective + H_C^DWE-penalty) Coefficients:")
print("  J_total (Quadratic Terms):")
print(json.dumps({str(k): f"{v:.6f}" for k, v in J_coeffs_total.items()}, indent=2)[:500] + "...") # Limit output size
print("  h_total (Linear Terms):")
print(json.dumps({str(k): f"{v:.6f}" for k, v in h_coeffs_total.items()}, indent=2))
print("-" * 50)


# --- Handle best_portfolio (from brute-force) ---
best_portfolio = (None, None) # E_min, E_max
if N <= 20:
    try:
        from qokit.portfolio_optimization import portfolio_brute_force
        print("Calculating classical brute-force solution (may take time for N up to 20)...")
        original_po_problem_for_brute_force = {
            "N":N, "K":K, "q":q, "means":mu, "cov":Sigma, "scale": 1.0
        }
        best_portfolio = portfolio_brute_force(original_po_problem_for_brute_force, return_bitstring=False)
        print(f"Brute-force classical optimal energy (E_min): {best_portfolio[0]:.6f}")
        print(f"Brute-force classical worst energy (E_max): {best_portfolio[1]:.6f}")
    except ImportError:
        print("qokit.portfolio_optimization.portfolio_brute_force not available or failed.")
        print("Approximation Ratio will not be calculated.")
    except Exception as e:
        print(f"Brute-force calculation failed: {e}")
        print("Approximation Ratio will not be calculated.")
else:
    print(f"Brute-force calculation is not feasible for N = {N} > 20. Approximation Ratio will not be calculated.")

print("-" * 50)


# --- Example QAOA Circuit Visualization and Gate Operations ---
print("\n--- Example QAOA Circuit and Gate Operations ---")
# Use dummy parameters for visualization (e.g., all zeros)
dummy_gammas = np.zeros(p)
dummy_betas = np.zeros(p)
example_circuit = get_parameterized_qaoa_circuit(
    po_problem=po_problem,
    depth=p,
    gamma=dummy_gammas,
    beta=dummy_betas
)
example_circuit.measure_all() # Add measurements for a complete circuit representation

print(f"\nLogical Circuit (before transpilation):")
print(f"  Circuit depth: {example_circuit.depth()}")
print(f"  Number of qubits: {example_circuit.num_qubits}")
print("  Gate counts (logical gates):")
print(example_circuit.count_ops())

# Transpile the circuit for a generic simulator to see how it would look on a real backend
temp_simulator = AerSimulator()
transpiled_example_circuit = transpile(example_circuit, temp_simulator, optimization_level=0) # opt_level 0 for direct mapping

print(f"\nTranspiled Circuit (optimization_level=0):")
print(f"  Transpiled circuit depth: {transpiled_example_circuit.depth()}")
print(f"  Transpiled gate counts:")
print(transpiled_example_circuit.count_ops())

# Draw the circuit (will be displayed inline in Jupyter)
print("\nVisualizing an Example QAOA Circuit (with dummy parameters and measurements):")
%matplotlib inline
fig = example_circuit.draw(output='mpl', style={'fontsize': 8, 'linecolor': '#000000'}, fold=100)
plt.tight_layout()
plt.show(fig)

print("-" * 50)


po_problem_for_worker = po_problem

# --- Generate Initial Points for Multi-Start ---
initial_points = []
for _ in range(num_parallel_runs):
    # For p=3, need 3 gammas and 3 betas
    gammas_initial = np.random.uniform(0, 2 * np.pi, p)
    betas_initial = np.random.uniform(0, np.pi, p)
    initial_points.append(tuple(np.concatenate((gammas_initial, betas_initial))))

# --- Parallel Execution ---
if __name__ == "__main__":
    print(f"Starting {num_parallel_runs} parallel QAOA optimization runs with p={p}...")
    start_overall_time = time.perf_counter()

    worker_args = [(initial_point, po_problem_for_worker, p, max_iterations_optimizer, num_shots_simulator, i + 1)
                   for i, initial_point in enumerate(initial_points)]

    all_results = []
    with Pool(processes=os.cpu_count()) as pool:
        results = [pool.apply_async(run_single_optimization, args) for args in worker_args]

        for future in results:
            all_results.append(future.get())

    end_overall_time = time.perf_counter()
    overall_duration = end_overall_time - start_overall_time
    print(f"\nAll parallel optimization runs completed in {overall_duration:.2f} seconds.")

    # --- Process and Summarize Results ---
    best_overall_energy = float('inf')
    best_overall_params = None
    best_overall_result_details = None

    print("\n--- Summary of All Individual Runs ---")
    num_successful_runs = 0

    all_results.sort(key=lambda x: x['optimal_energy'])

    for result in all_results:
        success_status = "SUCCESS" if result['success'] else "FAILED"
        print(f"Run {result['run_id']}: Energy={result['optimal_energy']:.6f}, Status={success_status}, Time={result['runtime_seconds']:.2f}s, NFEV={result['nfev']}, Message='{result['message']}'")

        if result['success']:
            num_successful_runs += 1

        if result['optimal_energy'] < best_overall_energy:
            best_overall_energy = result['optimal_energy']
            best_overall_params = result['optimal_params']
            best_overall_result_details = result

    print(f"\nTotal successful optimization runs: {num_successful_runs}/{num_parallel_runs}")

    if best_overall_params is not None:
        print("\n" + "---" * 20)
        print("### Best Overall Result Found from Multi-Start (Lowest Energy) ###")
        print(f"Best overall energy found: {best_overall_energy:.6f}")
        print(f"Best overall parameters: {np.round(best_overall_params, 3)}")
        print(f"This result came from Run ID: {best_overall_result_details['run_id']}")

        # --- Calculate and Display Approximation Ratio ---
        if best_portfolio[0] is not None and best_portfolio[1] is not None:
            E_min_classical = best_portfolio[0]
            E_max_classical = best_portfolio[1]

            # Avoid division by zero if E_max_classical == E_min_classical (unlikely for problems with variation)
            if (E_max_classical - E_min_classical) != 0:
                approximation_ratio = (best_overall_energy - E_min_classical) / (E_max_classical - E_min_classical)
                print(f"\nClassical Optimal Energy (E_min): {E_min_classical:.6f}")
                print(f"Classical Worst Energy (E_max): {E_max_classical:.6f}")
                print(f"Approximation Ratio (AR): {approximation_ratio:.4f} (Closer to 0 is better)")
            else:
                print("\nCannot calculate Approximation Ratio: Classical E_max - E_min is zero.")
        else:
            print("\nApproximation Ratio not calculated: Brute-force classical solution not available.")

    else:
        print("\n" + "---" * 20)
        print("No best overall result could be determined (all runs may have failed or no results were processed).")

--- Problem Parameters & Custom QUBO Defined for N=5 ---
Number of assets (N): 5
Assets to select (K): 2
QAOA Layers (p): 100
DWE Penalty Lambda_sum: 0
Optimizer Max Iterations: 2200
Simulator Shots: 256

--- Calculated QUBO Coefficients ---

Objective Hamiltonian (H_C^objective) Coefficients:
  J_obj (Quadratic Terms):
{
  "(0, 1)": "0.000388",
  "(0, 2)": "0.002087",
  "(0, 3)": "0.002373",
  "(0, 4)": "0.002863",
  "(1, 2)": "0.002239",
  "(1, 3)": "0.000866",
  "(1, 4)": "0.000918",
  "(2, 3)": "0.001517",
  "(2, 4)": "0.001660",
  "(3, 4)": "0.002085"
}...
  h_obj (Linear Terms):
{
  "0": "-0.052068",
  "1": "-0.093856",
  "2": "-0.078231",
  "3": "-0.068508",
  "4": "-0.035870"
}

Full Cost Hamiltonian (H_C = H_C^objective + H_C^DWE-penalty) Coefficients:
  J_total (Quadratic Terms):
{
  "(0, 1)": "0.000388",
  "(0, 2)": "0.002087",
  "(0, 3)": "0.002373",
  "(0, 4)": "0.002863",
  "(1, 2)": "0.002239",
  "(1, 3)": "0.000866",
  "(1, 4)": "0.000918",
  "(2, 3)": "0.001517",
  "(2

--------------------------------------------------
Starting 10 parallel QAOA optimization runs with p=100...


TypeError: run_single_optimization() missing 1 required positional argument: 'transpilation_level'

## 3. Parallel Multi-Start Optimization

This section sets up and executes multiple independent optimization runs in parallel using `ProcessPoolExecutor`. Each run will try to find a good QAOA parameter set from a different random starting point.

In [ ]:
# Assuming 'all_results' is still in your Jupyter Notebook's memory from the previous run
# You can verify this by typing 'print(len(all_results))' in a cell and running it.

best_overall_energy = float('inf')
best_overall_params = None

print("\n--- Reprocessing All Stored Run Results ---")
for result in all_results:
    # Print results of each run for inspection
    print(f"Run {result['run_id']}: Energy={result['optimal_energy']:.6f}, Success={result['success']}, Time={result['runtime_seconds']:.2f}s, NFEV={result['nfev']}")

    # Check for the lowest energy, regardless of scipy.optimize.minimize's 'success' flag
    if result['optimal_energy'] < best_overall_energy:
        best_overall_energy = result['optimal_energy']
        best_overall_params = result['optimal_params']

if best_overall_params is not None:
    print("\n" + "---" * 20)
    print("### Best Overall Result Found from Stored Data ###")
    print(f"Best overall energy found: {best_overall_energy:.6f}")
    print(f"Best overall parameters: {best_overall_params}")
else:
    print("\n" + "---" * 20)
    print("No best overall result could be determined (all_results might be empty or invalid).")

# Now, try running your Section 4 code after executing this block.


## 4. Optional: Analyze the Best Optimal Bitstring

This section takes the `optimal_params` from the best parallel run, samples the QAOA circuit, and evaluates the classical energy of the most probable bitstring. This helps to understand the practical outcome of the optimized QAOA parameters.

In [ ]:
if best_overall_params is not None: # <<< Make this the first line after comments/description
    print("\n" + "---" * 20)
    print("### Analyzing the Single Best Result for Optimal Bitstring ###")

    optimal_parameters = np.array(best_overall_params)
    print(f"Using optimal parameters from best multi-start run: {optimal_parameters}")

    # --- Extracting the Optimal Bitstring (Measurement) ---
    gamma_opt = ParameterVector('gamma', p)
    beta_opt = ParameterVector('beta', p)

    optimal_circuit = get_parameterized_qaoa_circuit(
        po_problem=po_problem,
        depth=p,
        ini_type='dicke',
        mixer_type='trotter_ring',
        T=1,
        simulator=None, # Use the Sampler directly for measurement
        mixer_topology='linear',
        gamma=gamma_opt,
        beta=beta_opt
    )

    optimal_circuit_bound = optimal_circuit.assign_parameters({
        gamma_opt: optimal_parameters[:p],
        beta_opt: optimal_parameters[p:]
    })

    optimal_circuit_bound.measure_all()

    sampler_shots = 10

    sampler = Sampler()

    print(f"\n--- Sampling for Optimal Bitstring ({sampler_shots} shots) ---")
    sampler_job = sampler.run(optimal_circuit_bound, shots=sampler_shots)
    sampler_result = sampler_job.result()

    counts = sampler_result.quasi_dists[0].binary_probabilities()
    classical_counts = {k: v for k, v in counts.items()}

    sorted_counts = sorted(classical_counts.items(), key=lambda item: item[1], reverse=True)

    print("Top 5 most frequent bitstrings and their probabilities:")
    optimal_bitstring = None
    for i, (bitstring, probability) in enumerate(sorted_counts[:5]):
        print(f"  {bitstring}: {probability:.4f}")
        if i == 0:
            optimal_bitstring = bitstring

    if optimal_bitstring:
        optimal_selection = np.array([int(b) for b in optimal_bitstring])
        selected_assets_count = np.sum(optimal_selection)

        print(f"\nMost probable portfolio selection (bitstring): {optimal_bitstring}")
        print(f"Number of selected assets: {selected_assets_count} (Expected K={K})")

        # Function to evaluate the classical objective energy for a given bitstring
        def evaluate_bitstring_energy(bitstring_array, original_mu, original_Sigma, original_q):
            x = bitstring_array
            portfolio_variance = np.dot(x, np.dot(original_Sigma, x))
            portfolio_return = np.dot(original_mu, x)
            return original_q * portfolio_variance - portfolio_return

        optimal_classical_energy_for_bitstring = evaluate_bitstring_energy(
            optimal_selection,
            po_problem['means'],
            po_problem['cov'],
            po_problem['q_orig']
        )
        print(f"Classical objective energy for the most probable bitstring: {optimal_classical_energy_for_bitstring:.6f}")
        print(f"Number of assets selected by most probable bitstring: {np.sum(optimal_selection)}")
    else:
        print("Could not determine optimal bitstring.")
